# Day 1: Python Essentials + LangChain Basics

**Learning Goals:**
- Python basics (compared to JavaScript)
- First LangChain chain
- LangSmith observability setup
- Build a simple chatbot

**Time:** 2-3 hours

## Part 1: Python Essentials for JS Developers

Quick comparison of Python vs JavaScript concepts you already know.

In [2]:
# 1. VARIABLES & TYPES
# Python uses snake_case (not camelCase)
user_name = "Rohan"  # No 'const' or 'let', just assign
age = 25
is_active = True  # Capitalized: True/False (not true/false)

print(f"Name: {user_name}, Age: {age}")  # f-strings are like template literals

# Type hints (optional, but helpful)
def greet(name: str) -> str:
    return f"Hello, {name}!"

print(greet("World"))

Name: Rohan, Age: 25
Hello, World!


In [3]:
# 2. LISTS vs ARRAYS
# Lists are like JS arrays
numbers = [1, 2, 3, 4, 5]
print(numbers[0])  # Same indexing
print(numbers[-1])  # Negative indexing (last item) - Python special!

# List comprehension (very Pythonic)
# JS: const doubled = numbers.map(n => n * 2)
doubled = [n * 2 for n in numbers]
print(doubled)

# Filtering
# JS: const evens = numbers.filter(n => n % 2 === 0)
evens = [n for n in numbers if n % 2 == 0]
print(evens)

1
5
[2, 4, 6, 8, 10]
[2, 4]


In [4]:
# 3. DICTIONARIES vs OBJECTS
# Dict is like JS object
user = {
    "name": "Rohan",
    "age": 25,
    "skills": ["JavaScript", "Node.js", "AWS"]
}

print(user["name"])  # Use brackets, not dot notation
print(user.get("email", "not provided"))  # Safe access with default

# Looping through dict
# JS: for (const [key, value] of Object.entries(user))
for key, value in user.items():
    print(f"{key}: {value}")

Rohan
not provided
name: Rohan
age: 25
skills: ['JavaScript', 'Node.js', 'AWS']


In [9]:
# 4. FUNCTIONS
# Similar to JS, but with some differences

# Default parameters
def create_message(text: str, priority: str = "normal") -> dict:
    return {
        "text": text,
        "priority": priority
    }

print(create_message("Hello"))
print(create_message("Urgent!", priority="high"))

# **kwargs (like spreading in JS)
def log_data(**kwargs):
    for key, value in kwargs.items():
        print(f"{key} = {value}")

log_data(user="Rohan", action="login", status="success")

{'text': 'Hello', 'priority': 'normal'}
{'text': 'Urgent!', 'priority': 'high'}
user = Rohan
action = login
status = success


In [10]:
# 5. CLASSES (you know this from TypeScript)
class User:
    def __init__(self, name: str, email: str):  # Constructor
        self.name = name  # 'self' is like 'this'
        self.email = email
        self._token = None  # _ prefix = private by convention
    
    def get_info(self) -> str:
        return f"{self.name} ({self.email})"
    
    def authenticate(self, password: str) -> bool:
        # Simplified example
        if password == "secret":
            self._token = "abc123"
            return True
        return False

user = User("Rohan", "rohan@example.com")
print(user.get_info())
print(user.authenticate("secret"))

Rohan (rohan@example.com)
True


## Part 2: Environment Setup

Load environment variables and set up our API connections.

In [1]:
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Get API keys
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

# Check if key exists
if not OPENROUTER_API_KEY:
    print("⚠️  Please set OPENROUTER_API_KEY in your .env file")
else:
    print("✅ OpenRouter API key loaded")

# Optional: LangSmith for observability
LANGCHAIN_TRACING = os.getenv("LANGCHAIN_TRACING_V2", "false") == "true"
if LANGCHAIN_TRACING:
    print("✅ LangSmith tracing enabled")
else:
    print("ℹ️  LangSmith tracing disabled (set LANGCHAIN_TRACING_V2=true to enable)")

✅ OpenRouter API key loaded
ℹ️  LangSmith tracing disabled (set LANGCHAIN_TRACING_V2=true to enable)


## Part 3: Your First LangChain Chain

Let's build a simple LLM chain using OpenRouter.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize LLM with OpenRouter
# OpenRouter uses OpenAI-compatible API
llm = ChatOpenAI(
    model="openai/gpt-3.5-turbo",  # or any model on OpenRouter
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.7,
)

print("✅ LLM initialized with OpenRouter")

NameError: name 'OPENROUTER_API_KEY' is not defined

In [3]:
# Simple direct call (like calling the API directly)
messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="What is LangChain?")
]

response = llm.invoke(messages)
print(response.content)

LangChain is a blockchain-based platform that aims to provide language learning services using blockchain technology. It leverages the decentralized nature of blockchain to offer secure and transparent language learning solutions. Users can access language courses, connect with tutors, and track their progress through the LangChain platform. The platform also uses smart contracts to facilitate payments and ensure trust between users.


### Using Prompt Templates

Prompt templates help you create reusable prompts with variables.

In [4]:
# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant specializing in {language}."),
    ("human", "{question}")
])

# Create a chain: prompt -> llm
chain = prompt | llm  # The | operator chains components

# Use the chain
response = chain.invoke({
    "language": "Python",
    "question": "How do I read a JSON file?"
})

print(response.content)

To read a JSON file in Python, you can use the `json` module which is included in the standard library. Here's a simple example:

```python
import json

# Open the JSON file in read mode
with open('data.json', 'r') as file:
    data = json.load(file)

# Now you can access the data as a Python dictionary
print(data)
```

In this example, replace `'data.json'` with the path to your JSON file. The `json.load()` function reads the JSON file and returns its contents as a Python dictionary. You can then work with this dictionary in your Python code.


### Adding Output Parsers

Output parsers help you get structured data from LLM responses.

In [5]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define output structure (like TypeScript interfaces!)
class CodeSuggestion(BaseModel):
    code: str = Field(description="The code snippet")
    explanation: str = Field(description="Brief explanation")
    language: str = Field(description="Programming language")

# Create parser
parser = PydanticOutputParser(pydantic_object=CodeSuggestion)

# Update prompt to include format instructions
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a coding assistant. {format_instructions}"),
    ("human", "Show me how to {task} in {language}")
])

# Create chain: prompt -> llm -> parser
chain = prompt | llm | parser

# Use it
result = chain.invoke({
    "task": "read a JSON file",
    "language": "Python",
    "format_instructions": parser.get_format_instructions()
})

print(f"Code:\n{result.code}\n")
print(f"Explanation: {result.explanation}")

Code:
import json

# Open the JSON file
with open('data.json', 'r') as file:
    data = json.load(file)

# Print the contents of the JSON file
print(data)

Explanation: The code snippet demonstrates how to read a JSON file in Python using the 'json' module. It opens a JSON file named 'data.json', reads its contents, and then prints the data stored in the file. Make sure to replace 'data.json' with the actual file path.


## Part 4: Building a Simple Chatbot

Let's create an interactive chatbot function.

In [ ]:
def simple_chatbot(user_input: str, system_prompt: str = "You are a helpful assistant.") -> str:
    """
    Simple chatbot function.
    
    Args:
        user_input: The user's question
        system_prompt: System instructions for the bot
    
    Returns:
        The bot's response
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"input": user_input})
    
    return response.content

# Test it
response = simple_chatbot(
    "Explain what a vector database is in simple terms.",
    system_prompt="You are a teacher who explains complex topics simply."
)

print(response)

Sure! A vector database is like a giant file cabinet where information is stored in a way that makes it easy to find and use. Instead of just storing data like words or numbers, a vector database also remembers the relationships between different pieces of information. This helps us quickly search for and retrieve the specific information we need, kind of like having a very organized and efficient library for data.


## Part 5: Observability with LangSmith

LangSmith helps you debug and monitor your chains.

In [ ]:
# If you have LangSmith configured, your chains are automatically traced!
# Let's make a few calls to see them in LangSmith

test_questions = [
    "What is the difference between a list and a tuple in Python?",
    "Explain async/await in simple terms.",
    "What are the benefits of using type hints?"
]

print("Running test queries...\n")
for i, question in enumerate(test_questions, 1):
    print(f"Q{i}: {question}")
    response = simple_chatbot(question)
    print(f"A{i}: {response[:100]}...\n")

print("✅ Check your LangSmith dashboard to see traces!")
print("   https://smith.langchain.com/")

## 🎯 Day 1 Exercises

Try these exercises to practice what you learned:

### Exercise 1: Custom Chatbot
Create a chatbot that acts as a code reviewer. It should:
- Accept code snippets
- Return suggestions for improvement
- Rate the code quality (1-10)

### Exercise 2: Structured Output
Create a chain that extracts key information from a meeting transcript:
- Attendees
- Action items
- Decisions made
Use Pydantic models for the output structure.

### Exercise 3: Model Comparison
Try the same prompt with different models on OpenRouter:
- `openai/gpt-3.5-turbo`
- `anthropic/claude-2`
- `meta-llama/llama-2-70b-chat`

Compare the responses and trace them in LangSmith.

In [ ]:
# Exercise 1: Your code here
# Hint: Create a CodeReview Pydantic model with fields for suggestions and rating

In [ ]:
# Exercise 2: Your code here
# Hint: Define a MeetingNotes model with List[str] for attendees and action_items

In [ ]:
# Exercise 3: Your code here
# Hint: Create a list of model names and loop through them

## 📝 Day 1 Summary

**What you learned:**
- ✅ Python essentials (compared to JavaScript)
- ✅ LangChain basic concepts: LLMs, Prompts, Chains
- ✅ Prompt templates with variables
- ✅ Output parsers with Pydantic
- ✅ LangSmith observability setup
- ✅ Built your first chatbot

**Key concepts:**
- **Chain**: Components connected with `|` operator (prompt | llm | parser)
- **Invoke**: Method to run chains with input
- **Pydantic Models**: Type-safe data structures (like TypeScript interfaces)
- **Tracing**: LangSmith automatically tracks all chain executions

**Next up (Day 2):**
- Different chain types (Sequential, Router)
- Conversation memory
- Context management
- Building a stateful chatbot

**Resources:**
- [LangChain Python Docs](https://python.langchain.com/docs/get_started/introduction)
- [OpenRouter Models](https://openrouter.ai/models)
- [Pydantic Tutorial](https://docs.pydantic.dev/latest/)

Great job! 🎉